In [ ]:
# 1. Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

# 2. Specify the path where the ZIP file is located (after uploading to Colab)
zip_file_path = '/content/drive/MyDrive/iu_xray.zip'  # Adjust this to match the exact path of your uploaded file

# 3. Specify the destination folder in Google Drive
destination_folder = '/content/'  # Replace this with your desired Google Drive path

# 4. Extract the ZIP file
import zipfile
import os

# Check if destination folder exists, create it if not
os.makedirs(destination_folder, exist_ok=True)

# Extract the ZIP file to the destination folder
with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
    zip_ref.extractall(destination_folder)

print(f"ZIP file extracted to: {destination_folder}")


Mounted at /content/drive
ZIP file extracted to: /content/


In [ ]:

# 2. Specify the path where the ZIP file is located (after uploading to Colab)
zip_file_path = '/content/R2GenCMN_Change.zip'  # Adjust this to match the exact path of your uploaded file

# 3. Specify the destination folder in Google Drive
destination_folder = '/content/'  # Replace this with your desired Google Drive path

# 4. Extract the ZIP file
import zipfile
import os

# Check if destination folder exists, create it if not
os.makedirs(destination_folder, exist_ok=True)

# Extract the ZIP file to the destination folder
with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
    zip_ref.extractall(destination_folder)

print(f"ZIP file extracted to: {destination_folder}")


ZIP file extracted to: /content/


In [ ]:
!pip install rouge_score pycocoevalcap==1.2

  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 104.3/104.3 MB 8.3 MB/s eta 0:00:00
  Created wheel for rouge_score: filename=rouge_score-0.1.2-py3-none-any.whl size=24935 sha256=33fd670a6501b13f403eef465216f5ec33194e7e02ac6d9ad0270e90b95358a3
  Stored in directory: /root/.cache/pip/wheels/5f/dd/89/461065a73be61a532ff8599a28e9beef17985c9e9c31e541b4
Successfully built rouge_score


In [ ]:
import os
import torch
import numpy as np
from google.colab import drive

# Set random seeds for reproducibility
torch.manual_seed(7580)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False
np.random.seed(7580)

# Set the dataset paths
image_dir = 'iu_xray/images/'  # updated path as per new values
ann_path = 'iu_xray/annotation.json'  # updated path as per new values

# Make sure the required directories exist
os.makedirs('/content/drive/MyDrive/results/iu_xray', exist_ok=True)

# Import the necessary modules (ensure they are uploaded to Colab or in the appropriate paths)
import sys
# sys.path.append('/content/modules')  # Update this with the actual path if needed

# Import functions and classes
from models.models import BaseCMNModel
from modules.dataloaders import R2DataLoader
from modules.loss import compute_loss
from modules.metrics import compute_scores
from modules.optimizers import build_optimizer, build_lr_scheduler
from modules.tokenizers import Tokenizer
from modules.trainer import Trainer

# Define the args manually (equivalent to argparse in the original script)
class Args:
    def __init__(self):
        self.image_dir = 'iu_xray/images/'  # updated default
        self.ann_path = 'iu_xray/annotation.json'  # updated default
        self.dataset_name = 'iu_xray'  # updated default
        self.max_seq_length = 30  # updated default which was 60
        self.threshold = 3  # updated default
        self.num_workers =  2  # default value (unchanged)
        self.batch_size = 8 # updated default which was 16
        self.visual_extractor = 'resnet101'  # default value (unchanged)
        self.visual_extractor_pretrained = True  # default value (unchanged)
        self.d_model = 512  # default value (unchanged)
        self.d_ff = 512  # default value (unchanged)
        self.d_vf = 2048  # default value (unchanged)
        self.num_heads = 8  # default value (unchanged)
        self.num_layers = 4 # updated default which was 3
        self.dropout = 0.1  # default value (unchanged)
        self.logit_layers = 1  # default value (unchanged)
        self.bos_idx = 0  # default value (unchanged)
        self.eos_idx = 0  # default value (unchanged)
        self.pad_idx = 0  # default value (unchanged)
        self.use_bn = 0  # default value (unchanged)
        self.drop_prob_lm = 0.5  # default value (unchanged)
        self.topk = 32  # updated default
        self.cmm_size = 1024  # updated default value which was 2048
        self.cmm_dim = 512  # updated default
        self.sample_method = 'beam_search'  # default value (unchanged)
        self.beam_size = 3  # updated default
        self.temperature = 1.0  # default value (unchanged)
        self.sample_n = 1  # default value (unchanged)
        self.group_size = 1  # default value (unchanged)
        self.output_logsoftmax = 1  # default value (unchanged)
        self.decoding_constraint = 0  # default value (unchanged)
        self.block_trigrams = 1  # default value (unchanged)
        self.n_gpu = 1  # default value (unchanged)
        self.epochs = 100  # updated default
        self.save_dir = '/content/drive/MyDrive/results/iu_xray'  # updated default
        self.resume='/content/drive/MyDrive/results/iu_xray/current_checkpoint.pth'
        self.record_dir = 'records/'  # default value (unchanged)
        self.log_period = 50  # updated default
        self.save_period = 1  # default value (unchanged)
        self.monitor_mode = 'max'  # default value (unchanged)
        self.monitor_metric = 'BLEU_4'  # default value (unchanged)
        self.early_stop = 50  # default value (unchanged)
        self.optim = 'Adam'  # default value (unchanged)
        self.lr_ve = 1e-4  # updated default
        self.lr_ed = 5e-4  # updated default
        self.step_size = 10  # updated default
        self.gamma = 0.8  # updated default
        self.adam_betas = (0.9, 0.98)  # default value (unchanged)
        self.adam_eps = 1e-9  # default value (unchanged)
        self.amsgrad = True  # default value (unchanged)
        self.noamopt_warmup = 5000  # default value (unchanged)
        self.noamopt_factor = 1  # default value (unchanged)
        self.lr_scheduler = 'StepLR'  # default value (unchanged)
        self.seed = 7580  # updated default
        self.resume = None  # default value (unchanged)
        self.weight_decay= 5e-5

args = Args()

# Create tokenizer
tokenizer = Tokenizer(args)

# Data loader setup
train_dataloader = R2DataLoader(args, tokenizer, split='train', shuffle=True)
val_dataloader = R2DataLoader(args, tokenizer, split='val', shuffle=False)
test_dataloader = R2DataLoader(args, tokenizer, split='test', shuffle=False)

# Build model architecture
model = BaseCMNModel(args, tokenizer)

# Set up the loss function and metrics
criterion = compute_loss
metrics = compute_scores

# Set up the optimizer and learning rate scheduler
optimizer = build_optimizer(args, model)
lr_scheduler = build_lr_scheduler(args, optimizer)

# Specify the mode you want to run:
mode = 'train'  # Change to 'test' or 'eval' based on what you need to do

# Create trainer and start training/evaluating
trainer = Trainer(model, criterion, metrics, optimizer, args, lr_scheduler, train_dataloader, val_dataloader, test_dataloader)

# If you're training, you would call:
if mode == 'train':
    trainer.train()

# If you're testing, you would call:
elif mode == 'test':
    trainer.test()


/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet101_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet101_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
INFO:modules.trainer:Loading checkpoint: /content/drive/MyDrive/results/iu_xray/current_checkpoint.pth ...


Loading checkpoint: /content/drive/MyDrive/results/iu_xray/current_checkpoint.pth ...


/content/modules/trainer.py:193: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  checkpoint = torch.load(resume_path)


RuntimeError: Attempting to deserialize object on a CUDA device but torch.cuda.is_available() is False. If you are running on a CPU-only machine, please use torch.load with map_location=torch.device('cpu') to map your storages to the CPU.

In [ ]:
import pycocoevalcap.bleu
print(dir(pycocoevalcap.bleu))


['__doc__', '__file__', '__loader__', '__name__', '__package__', '__path__', '__spec__']


In [ ]:
print("hello")